In [164]:
import tensorflow as tf
import numpy as np
import os
import cv2
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tf.compat.v1.disable_eager_execution()

Funcionalidade das bibliotecas que foram utilizadas:

- Numpy: Facilita algumas operações e manipulações matematicas, e permite a captura da forma vetorial das imagens.
- os: Fornece algumas funções de manipulação do proprio sistema operacional que foram uteis para testes feitos antes da integração com a base de dados.
- cv2: Permite o redimensionamento das imagens.
- sklearn: Biblioteca de aprendizagem de maquina da qual foi utilizada as funções que implementaram o perceptron
- TensorFlow





Considerando uma implementação através de manipulação de arquivos para a base de dados:

É pego a lista de diretórios do caminho atual, e então atribuido a uma lista. Esses diretórios são as categorias de veiculos que serão avaliadas, no caso, Aviões e Barcos.

In [184]:
vehicle_types = os.listdir('Dataset/imgs//')
vehicle_types

['airplanes', 'ship']

Então, para cada diretório que foi descoberto é feita uma varredura, e atribuida cada imagem que está no formato jpg para a lista de imagens, e a sua categoria correspondente para a lista de labels.

In [189]:
path = 'Dataset/imgs//'

sizeExpected = 16

images = []
labels = []

for i in vehicle_types:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path) if i.endswith('.jpg')]
    print(filenames)
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img,(sizeExpected, sizeExpected))
        images.append(img)
        labels.append(i)

['airplane12.jpg', 'airplane18.jpg', 'airplane24.jpg', 'airplane30.jpg', 'airplane31.jpg', 'airplane32.jpg', 'airplane33.jpg', 'airplane34.jpg', 'airplane35.jpg', 'airplane36.jpg', 'airplane37.jpg', 'airplane38.jpg', 'airplane39.jpg', 'airplane40.jpg', 'airplane41.jpg', 'airplane42.jpg', 'airplane43.jpg', 'airplane44.jpg', 'airplane45.jpg', 'airplane46.jpg', 'airplane47.jpg', 'airplane48.jpg', 'airplane49.jpg', 'airplane50.jpg', 'airplane6.jpg']
['1937807.jpg', '1951775.jpg', '1958504.jpg', '1982219.jpg', '1996135.jpg', '2000408.jpg', '2002178.jpg', '2003231.jpg', '2004193.jpg', '2004538.jpg', '2005202.jpg', '2005205.jpg', '2005208.jpg', '2006104.jpg', '2007300.jpg', '2008187.jpg', '2010126.jpg', '2010303.jpg', '2010304.jpg', '2012364.jpg', '2012365.jpg', '2012366.jpg', '2012371.jpg', '2012989.jpg', '2013004.jpg']


Com isso, pode-se converter as imagens em questão para sua forma vetorial, garantir que estejam no formato correto (Float32) e sobre a escala de cinza

Com a função shape podemos ver que o primeiro elemento da 4-tupla é a quantidade de imagens que foram pegas, os proximos dois são a dimensão que foi escolhida para análise, e o ultimo elemento indica que se trata de uma imagem colorida (RGB).

In [191]:
images = np.array(images)
images = images.astype('float32')/255
images.shape

(50, 16, 16)

Com as funções fornecidas pela biblioteca sklearn podemos começar a criação e atribuição de labels para o treinamento. Dessa forma, a variavel x recebe a lista de imagens pegas, e a y recebe a lista binária de labels (o que permite que as labels sejam "entendidas" pelo computador), onde no nosso caso, seria:
- 0 para Gatos
- 1 para Cachorros.

In [168]:
x = images
y = labels
y_labelEncoder = LabelEncoder()
y = y_labelEncoder.fit_transform(y)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


Nesse momento é separado as labels em duas classes já que precisamos de duas colunas de classificação.

In [169]:
y = y.reshape(-1,1)

ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
y = ct.fit_transform(y)
print(y)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


Neste momento é feita a separação entre treino e teste.

In [170]:
x,Y = shuffle(x,y, random_state=1)
train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.30, random_state=415)

In [171]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(70, 60, 60, 3)
(70, 2)
(30, 60, 60, 3)
(30, 2)


Como o objetivo é utilizar um perceptron para resolver o problema da classificação, precisamos transformar o formato da imagem para apenas uma dimensão. Dessa forma, temos que:

Nova dimensão = (n, r * r * 3)

Sendo o n o valor do primeiro elemento da 4-tupla da célula anterior (o numero de imagens do treino e do teste), e r o numéro que redimensionamos a imagem anteriormente.

Logo:
- Para o treino:    Nova dimensão = (3, 10800)
- Para o teste:     Nova dimensão = (1, 10800)

In [172]:
train_x=np.reshape(train_x,(70,10800))
print(train_x.shape)

test_x=np.reshape(test_x,(30,10800))
test_x.shape

(70, 10800)


(30, 10800)

In [173]:
learning_rate = 0.20
training_epochs = 1000
n_dim=60
n_class=2


Perceptron_x = tf.compat.v1.placeholder(tf.float32, [None, 10800])
Perceptron_W = tf.Variable(tf.zeros([10800,n_class]))
Perceptron_b = tf.Variable(tf.zeros([n_class]))
Perceptron_y = tf.compat.v1.placeholder(tf.float32,[None,n_class]) 

In [174]:
init = tf.compat.v1.global_variables_initializer()
pred = tf.nn.softmax(tf.matmul(Perceptron_x, Perceptron_W)+ Perceptron_b) # perceptron
 
error_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Perceptron_y)) 
training_step = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(error_function)


In [175]:

sess = tf.compat.v1.Session()
sess.run(init)

cost_history = []

In [176]:
for epoch in range(training_epochs):
    sess.run(training_step,feed_dict={Perceptron_x:train_x,Perceptron_y:train_y})
    cost = sess.run(error_function,feed_dict={Perceptron_x: train_x,Perceptron_y: train_y})
    cost_history = np.append(cost_history,cost)
    print('epoch : ', epoch,  ' - ', 'cost: ', cost)


epoch :  0  -  cost:  0.7705651
epoch :  1  -  cost:  0.7705628
epoch :  2  -  cost:  0.7705606
epoch :  3  -  cost:  0.7705584
epoch :  4  -  cost:  0.7705563
epoch :  5  -  cost:  0.77055424
epoch :  6  -  cost:  0.7705522
epoch :  7  -  cost:  0.77055025
epoch :  8  -  cost:  0.7705484
epoch :  9  -  cost:  0.77054656
epoch :  10  -  cost:  0.7705447
epoch :  11  -  cost:  0.770543
epoch :  12  -  cost:  0.77054125
epoch :  13  -  cost:  0.7705396
epoch :  14  -  cost:  0.770538
epoch :  15  -  cost:  0.77053636
epoch :  16  -  cost:  0.7705348
epoch :  17  -  cost:  0.7705333
epoch :  18  -  cost:  0.77053183
epoch :  19  -  cost:  0.7705304
epoch :  20  -  cost:  0.7705289
epoch :  21  -  cost:  0.7705276
epoch :  22  -  cost:  0.7705262
epoch :  23  -  cost:  0.77052486
epoch :  24  -  cost:  0.77052355
epoch :  25  -  cost:  0.7705223
epoch :  26  -  cost:  0.77052104
epoch :  27  -  cost:  0.7705198
epoch :  28  -  cost:  0.77051866
epoch :  29  -  cost:  0.77051747
epoch :  30

In [181]:
image = load_img('test1.jpg', target_size=(60,60))
image = np.array(image)

image.shape
image = np.reshape(image,(1,10800))

In [183]:
result = sess.run(pred, feed_dict={Perceptron_x: image})
print(result)

if(result[0][0] > result[0][1]):
    print("Avião")
else:
    print("Barco")

[[0. 1.]]
Barco
